## CONFIGUARATION MANGER 

In [1]:
import os

In [2]:
%pwd

'/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [11]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

## dowNloading  zip files from google dive 

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix =  "https://drive.google.com/uc?id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



### file_id = dataset_url.split("/")[-2]

This line extracts the **Google Drive file ID** from a URL.

 Example

If the URL is:

```python
dataset_url = "https://drive.google.com/file/d/1a2b3c4d5e6f7g8h9/view?usp=sharing"
```

Then this line:

```python
file_id = dataset_url.split("/")[-2]
```

will do the following:

1. `dataset_url.split("/")`
   → Splits the string by slashes `/`, giving:
   `['https:', '', 'drive.google.com', 'file', 'd', '1a2b3c4d5e6f7g8h9', 'view?usp=sharing']`

2. `[-2]`
   → Picks the second-to-last element, which is the file ID:
   `'1a2b3c4d5e6f7g8h9'`

---

 Why `[-2]`?

* Google Drive links follow a pattern:
  `.../d/<file_id>/...`
  So `[-2]` gives the ID.



### gdown.download(prefix+file_id,zip_download_dir)

In [ ]:
prefix = "https://drive.google.com/uc?id="
file_id = "1a2b3c4d5e6f7g8h9"


## downloading ZIP  from google drive HANDLING ERROR WITH MIME 

In [19]:
import os, zipfile, logging, pathlib, mimetypes
import gdown
#from cnnClassifier.entity import DataIngestionConfig   # ← keep your import
logger = logging.getLogger(__name__)

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    # ------------ 1. download -----------------
    def download_file(self) -> str:
        """
        Downloads the file from Google-Drive link and returns local path.
        """
        url  = self.config.source_URL
        dest = self.config.local_data_file
        os.makedirs(os.path.dirname(dest), exist_ok=True)

        logger.info(f"⬇️  Downloading {url}  →  {dest}")
        gdown.download(url, dest, quiet=False, fuzzy=True)
        logger.info("✅  Download complete")

        return dest

    # ------------ 2. extract ------------------
    def extract_zip_file(self) -> None:
        """
        Safely extracts `*.zip` archive to `unzip_dir`.
        Raises ValueError if file is not a zip archive.
        """
        zip_path  = pathlib.Path(self.config.local_data_file)
        unzip_dir = pathlib.Path(self.config.unzip_dir)
        unzip_dir.mkdir(parents=True, exist_ok=True)

        # ✨ Extra guard – extension + MIME type
        mime, _ = mimetypes.guess_type(zip_path)
        if zip_path.suffix.lower() != ".zip" or mime not in ("application/zip", "application/x-zip-compressed"):
            msg = f"{zip_path} is not a zip archive"
            logger.error(msg)
            raise ValueError(msg)

        try:
            with zipfile.ZipFile(zip_path, "r") as zf:
                zf.extractall(unzip_dir)
            logger.info(f"📦  Extracted to {unzip_dir.resolve()}")
        except zipfile.BadZipFile as err:
            msg = f"Corrupted or non-zip file: {zip_path}"
            logger.exception(msg)
            raise ValueError(msg) from err


In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix =  "https://drive.google.com/uc?id="
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



## data INGESTION PIPELINE 

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-08 00:00:42,054: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-08 00:00:42,058: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-08 00:00:42,059: INFO: common: created directory at: artifacts]
[2025-06-08 00:00:42,060: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-08 00:00:42,061: INFO: 767000518: Downloading data from https://drive.google.com/file/d/1BDVprz9NtenCp3wDovA2lfKVBzJDCTt2/view?usp=sharing into file artifacts/data_ingestion/data_2.zip]


Downloading...
From (original): https://drive.google.com/uc?id=1BDVprz9NtenCp3wDovA2lfKVBzJDCTt2
From (redirected): https://drive.google.com/uc?id=1BDVprz9NtenCp3wDovA2lfKVBzJDCTt2&confirm=t&uuid=5428cea0-7c17-46d7-bb5b-06803a1ee85b
To: /Users/priyam/DIL_LAB/HAR_HEAT_IMAGEdataset/artifacts/data_ingestion/data_2.zip
100%|██████████| 1.50G/1.50G [10:30<00:00, 2.39MB/s]  

[2025-06-08 00:11:18,535: INFO: 767000518: Downloaded data from https://drive.google.com/file/d/1BDVprz9NtenCp3wDovA2lfKVBzJDCTt2/view?usp=sharing into file artifacts/data_ingestion/data_2.zip]
